In [2]:
import json
from dotenv import load_dotenv
from functools import reduce
import operator
import os
import pandas as pd
import pymongo
from pymongo import MongoClient
import requests

In [3]:
load_dotenv()

True

#### MELBOURNE CBD

In [37]:
melbourne_cbd = {'type': 'Point', 'coordinates': [-37.81227175879301,144.96262186884945]}

In [38]:
def getdata_fsq(url, city, venue):
    client_id = os.getenv("tok1")
    client_secret = os.getenv("tok2")
    url_ = f'{url}'
    c = city['coordinates'][0]
    c1 = city['coordinates'][1]
    parametros = {"client_id": client_id, "client_secret": client_secret, "v": "20180323", "ll": f'{c}, {c1}',
                  "query": venue, "limit": 55}
    resp = requests.get(url_, params=parametros).json()
    return resp

#### HACER UNA FUNCION PARA GENERAR DATAS POR CADA RUBRO, POR CADA CIUDAD

In [39]:
cathegories_m = ['vegan', 'airport', 'station', 'starbucks', 'veterinary', 'basketball', 'school']

In [9]:
def datagenerator(url, city, cathegories):
    cath = cathegories
    data_ = []
    for c in cath:
        data = getdata_fsq(url, city, c)
        data_.append(data['response']['venues'])
    return data_

In [10]:
melbourne = datagenerator('https://api.foursquare.com/v2/venues/search', melbourne_cbd, cathegories_m)

In [47]:
melbourne[0]

[{'id': '5d19912aaa7be7002a2256b9',
  'name': 'Home Vegan Bar',
  'contact': {},
  'location': {'address': '4/699.',
   'crossStreet': 'Collins st',
   'lat': -37.819958,
   'lng': 144.95122,
   'labeledLatLngs': [{'label': 'display',
     'lat': -37.819958,
     'lng': 144.95122}],
   'distance': 1318,
   'postalCode': '3008',
   'cc': 'AU',
   'city': 'Docklands',
   'state': 'VIC',
   'country': 'Australia',
   'formattedAddress': ['4/699. (Collins st)',
    'Docklands VIC 3008',
    'Australia']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
     'suffix': '.png'},
    'primary': True}],
  'verified': False,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'beenHere': {'count': 0,
   'lastCheckinExpiredAt': 0,
   'ma

In [18]:
def getFromDict(dic,map_):
    return reduce(operator.getitem, map_, dic)

def type_point(gps):
    return {"type": "Point", "coordinates": gps}

def jsonxport(name, l):
    n = f'{name}.json'
    with open(n, 'w') as f:
        json.dump(l, f)

#### OJO, ACA INVERTI EL ORDEN, LONG LAT

In [48]:
def jsongenerator(data, name):
    d = data
    map_name = ['name']
    map_categories = ['categories']
    map_lat = ['location', 'lat']
    map_long = ['location', 'lng']
    l = []
    for dic in d:
        ndic = {}
        ndic['name']= getFromDict(dic,map_name)
        ndic['categories']= getFromDict(dic,map_categories)
        ndic['latitude']= getFromDict(dic,map_lat)
        ndic['longitude']= getFromDict(dic,map_long)
        ndic['location']= type_point([ndic["longitude"],ndic["latitude"]])
        l.append(ndic)
    return jsonxport(name, l)

In [49]:
a = jsongenerator(melbourne[0], 'veganm')

In [50]:
b = jsongenerator(melbourne[1], 'airportm')

In [51]:
c = jsongenerator(melbourne[2], 'station')

In [52]:
d = jsongenerator(melbourne[3], 'starbucksm')

In [53]:
e = jsongenerator(melbourne[4], 'veterinary')

In [54]:
f = jsongenerator(melbourne[5], 'basketballm')

In [55]:
g = jsongenerator(melbourne[6], 'schoolsm')

#### llamada a la collection

In [56]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")

In [57]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [58]:
g = db.get_collection("transport")

In [59]:
g.find_one().keys()

dict_keys(['_id', 'name', 'categories', 'latitude', 'longitude', 'location'])

In [82]:
cond1 = {'categories.name': 'Train Station'}

In [86]:
cond2 = {"location": {"$near": {"$geometry": {'type': 'Point', 'coordinates': [144.96262186884945,-37.81227175879301]}, "$minDistance": 0, "$maxDistance": 2000}}}

In [87]:
query = {'$and': [cond1, cond2]}

In [88]:
list(g.find(query))

[{'_id': ObjectId('6134aa9f3681de6a29cda060'),
  'name': 'Melbourne Central Station Customer Service Centre',
  'categories': [{'id': '4bf58dd8d48988d129951735',
    'name': 'Train Station',
    'pluralName': 'Train Stations',
    'shortName': 'Train Station',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': -37.810314,
  'longitude': 144.962932,
  'location': {'type': 'Point', 'coordinates': [144.962932, -37.810314]}},
 {'_id': ObjectId('6134aa9f3681de6a29cda059'),
  'name': 'Melbourne Central Station',
  'categories': [{'id': '4bf58dd8d48988d129951735',
    'name': 'Train Station',
    'pluralName': 'Train Stations',
    'shortName': 'Train Station',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_',
     'suffix': '.png'},
    'primary': True}],
  'latitude': -37.81029379973877,
  'longitude': 144.96297447622638,
  'location': {'type': 'Point',
   'coo

In [283]:
#q1 = db.gastronomy.aggregate([{'$geoNear': {'near': paris_ld, 'distanceField': "distance", 'maxDistance': 1000, 'distanceMultiplier': 6371,
#                                   'spherical': True}}])